In [117]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
import csv

def clearData(data):
    # Taking reasonable data
    allDataFirst = [d for d in zip(data['orderDate'], data['deliveryDate']) if list(d)[1] != "?"  ]
    allDataAll = [list(d) for d in allDataFirst if datetime.strptime(list(d)[0], '%Y-%m-%d') < datetime.strptime(list(d)[1], '%Y-%m-%d')]

    # Calculating ship time
    shipTime = [(datetime.strptime(d[1], '%Y-%m-%d')-datetime.strptime(d[0], '%Y-%m-%d')) / timedelta (days=1) for d in allDataAll]
    shiptimeMean = np.mean(np.array(shipTime))

    # First changin ? with orderDate + Mean
    clearedDataFirst = [list(d) if list(d)[1] != "?" else [list(d)[0],(datetime.strptime(list(d)[0], '%Y-%m-%d') + timedelta(days=11)).strftime("%Y-%m-%d") ]  for d in zip(data['orderDate'], data['deliveryDate']) ]

    # Second changing 1990 with orderDate + Mean
    clearedTimeDataSecond = [d if datetime.strptime(list(d)[0], '%Y-%m-%d') < datetime.strptime(list(d)[1], '%Y-%m-%d') else [d[0],(datetime.strptime(list(d)[0], '%Y-%m-%d') + timedelta(days=11)).strftime("%Y-%m-%d")] for d in clearedDataFirst]

    # Changing data with new values
    data["orderDate"] = [row[0] for row in clearedTimeDataSecond]
    data["deliveryDate"] = [row[1] for row in clearedTimeDataSecond]

    # Calculating age mean
    allDataFirstAge = [d for d in data['dateOfBirth'] if d != "?"  ]
    allDataAllAge = [d for d in allDataFirstAge if datetime.strptime(d, '%Y-%m-%d') > datetime.strptime("1930-01-01", '%Y-%m-%d') and datetime.strptime(d, '%Y-%m-%d') < datetime.strptime("2000-01-01", '%Y-%m-%d')]
    ageYear = [((datetime.strptime("2014-01-01", '%Y-%m-%d')-datetime.strptime(d, '%Y-%m-%d'))/timedelta (days=1))/365 for d in allDataAllAge]
    ageYearMean = np.mean(np.array(ageYear))

    # Clearing the birthdate data
    allDataFirstAgeChanged = [d if d != "?" else (datetime.strptime("2014-01-01", '%Y-%m-%d') - timedelta(days=48)).strftime('%Y-%m-%d') for d in data['dateOfBirth']   ]
    allDataAllAgeChanged = [d if datetime.strptime(d, '%Y-%m-%d') > datetime.strptime("1930-01-01", '%Y-%m-%d') and datetime.strptime(d, '%Y-%m-%d') < datetime.strptime("2000-01-01", '%Y-%m-%d') else (datetime.strptime("2014-01-01", '%Y-%m-%d') - timedelta(days=48*365.2)).strftime('%Y-%m-%d') for d in allDataFirstAgeChanged]
    data["dateOfBirth"] = allDataAllAgeChanged

    colorDict = {}
    colorArary = ['denim', 'ocher', 'curry', 'green', 'black', 'brown', 'red', 'mocca',
     'anthracite', 'olive', 'petrol', 'blue', 'grey', 'beige', 'ecru', 'turquoise',
     'magenta', 'purple', 'pink', 'khaki', 'navy', 'habana', 'silver', 'white',
     'nature', 'stained', 'orange', 'azure', 'apricot', 'mango', 'berry', 'ash',
     'hibiscus', 'fuchsia', 'dark denim', 'mint', 'ivory', 'yellow',
     'bordeaux', 'pallid', 'ancient', 'baltic blue', 'almond', 'aquamarine',
     'aubergine','aqua', 'dark garnet', 'dark grey', 'avocado', 'creme',
     'champagner', 'cortina mocca', 'currant purple', 'cognac', 'aviator', 'gold',
     'ebony', 'cobalt blue', 'kanel','curled', 'caramel', 'antique pink',
     'darkblue', 'copper coin', 'terracotta', 'basalt', 'amethyst', 'coral', 'jade',
     'opal', 'striped', 'mahagoni', 'floral', 'dark navy', 'dark oliv',
     'vanille', 'ingwer', 'iron', 'graphite', 'leopard', 'bronze', 'crimson',
     'lemon', 'perlmutt']

    for index, x in enumerate(colorArary):
        colorDict[x] = index
    colorData = list(data['color'])
    for index, x in enumerate(colorData):
        if x  == "?":
            colorData[index] = "black"
        elif x  == "brwon":
            colorData[index] = "brown"
        elif x  == "blau":
            colorData[index] = "blue"
        elif x  == "oliv":
            colorData[index] = "olive"
        colorData[index] = colorDict[colorData[index]]
    data['color'] = colorData
    return data

In [128]:
data = clearData(pd.read_csv('train.txt', sep=",").sample(n=10020))


In [129]:
# Feature 1, if taken date > 10 1, else 0
shipmentDateF = np.array([ 0 if ((datetime.strptime(list(d)[1], '%Y-%m-%d')-datetime.strptime(list(d)[0], '%Y-%m-%d')) / timedelta (days=1)) <= 10 else 1 for d in zip(data['orderDate'], data['deliveryDate'])])

# Feature 2 - specialDateChrismass, if order is close to speacial date and came late it is 1
# Feature 3 - specialDateOthers, if order is close to speacial date and came late it is 1
dateRepresentationOfData = [ [(int('%02d' % datetime.strptime(list(d)[0], '%Y-%m-%d').day),int('%02d' % datetime.strptime(list(d)[0], '%Y-%m-%d').month)), (int('%02d' % datetime.strptime(list(d)[1], '%Y-%m-%d').day),int('%02d' % datetime.strptime(list(d)[1], '%Y-%m-%d').month))] for d in zip(data['orderDate'], data['deliveryDate'])]
specialDateChrismassF = np.array([1 if (d[0][0] > 14 and d[0][0] < 22 and d[0][1] == 12) and (d[1][0] >= 1 and d[1][1] >= 1) else 0 for d in dateRepresentationOfData])
specialDateOthersF = np.array([1 if (d[0][0] > 3 and d[0][0] < 11 and d[0][1] == 5) and (d[1][0] >= 15 and d[1][1] >= 5) or (d[0][0] > 8 and d[0][0] < 14 and d[0][1] == 2) and (d[1][0] >= 18 and d[1][1] >= 2) else 0 for d in dateRepresentationOfData])

# Feature 4, <30 - 0 , 30-55 - 1, 55> - 2
ageNummArray = [((datetime.strptime("2014-01-01", '%Y-%m-%d')-datetime.strptime(d, '%Y-%m-%d'))/timedelta (days=1))/365 for d in data['dateOfBirth']]
ageNumF = np.array([0 if d <= 30 else (1 if d <= 55 else 2) for d in ageNummArray])

# Feature 5 manufacturer id mean value
manuReturnRateF = np.array([d for d in data["manufacturerID"]])

# Feature 6 costumer id mean value
userReturnRateF = np.array([d for d in data["customerID"]])

# Feature 7 costumer id mean value
colorDataF = np.array(data['color'])

In [130]:
# Creating train matrix
trainMatrix = np.vstack((shipmentDateF, specialDateChrismassF, specialDateOthersF, ageNumF, manuReturnRateF, userReturnRateF, colorDataF)).T

# Creating label array
labelArray = np.array(data['returnShipment'])

logisticRegr = LogisticRegression()
logisticRegr.fit(trainMatrix, labelArray)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [131]:
# Test to Feature
testData =  clearData( pd.read_csv('test.txt', sep=","))

In [132]:
# Feature 1, if taken date > 10 1, else 0
shipmentDateTF = np.array([ 0 if ((datetime.strptime(list(d)[1], '%Y-%m-%d')-datetime.strptime(list(d)[0], '%Y-%m-%d')) / timedelta (days=1)) <= 10 else 1 for d in zip(testData['orderDate'], testData['deliveryDate'])])

# Feature 2 - specialDateChrismass, if order is close to speacial date and came late it is 1
# Feature 3 - specialDateOthers, if order is close to speacial date and came late it is 1
dateRepresentationOfDataT = [ [(int('%02d' % datetime.strptime(list(d)[0], '%Y-%m-%d').day),int('%02d' % datetime.strptime(list(d)[0], '%Y-%m-%d').month)), (int('%02d' % datetime.strptime(list(d)[1], '%Y-%m-%d').day),int('%02d' % datetime.strptime(list(d)[1], '%Y-%m-%d').month))] for d in zip(testData['orderDate'], testData['deliveryDate'])]
specialDateChrismassTF = np.array([1 if (d[0][0] > 14 and d[0][0] < 22 and d[0][1] == 12) and (d[1][0] >= 1 and d[1][1] >= 1) else 0 for d in dateRepresentationOfDataT])
specialDateOthersTF = np.array([1 if (d[0][0] > 3 and d[0][0] < 11 and d[0][1] == 5) and (d[1][0] >= 15 and d[1][1] >= 5) or (d[0][0] > 8 and d[0][0] < 14 and d[0][1] == 2) and (d[1][0] >= 18 and d[1][1] >= 2) else 0 for d in dateRepresentationOfDataT])

# Feature 4, <30 - 0 , 30-55 - 1, 55> - 2
ageNummArrayT = [((datetime.strptime("2014-01-01", '%Y-%m-%d')-datetime.strptime(d, '%Y-%m-%d'))/timedelta (days=1))/365 for d in testData['dateOfBirth']]
ageNumTF = np.array([0 if d <= 30 else (1 if d <= 55 else 2) for d in ageNummArrayT])

# Feature 5 manufacturer id mean value
manuReturnRateTF = np.array([d for d in testData["manufacturerID"]])

# Feature 6 costumer id mean value
userReturnRateTF = np.array([d for d in testData["manufacturerID"]])

# Feature 7 costumer id mean value
colorDataTF = np.array(testData['color'])

In [133]:
# Creating test matrix
testMatrix = np.vstack((shipmentDateTF, specialDateChrismassTF, specialDateOthersTF, ageNumTF, manuReturnRateTF, userReturnRateTF, colorDataTF)).T



In [134]:
predictions = logisticRegr.predict_proba(testMatrix)
print(sorted([e[1] for e in predictions]))

[0.39921340568138314, 0.39921340568138314, 0.39984440978672015, 0.399898080037538, 0.4002726835559396, 0.4002726835559396, 0.40047497961478484, 0.40047497961478484, 0.40052944487010494, 0.4008558697877189, 0.4009042447074329, 0.4009042447074329, 0.4009042447074329, 0.4009042447074329, 0.4009042447074329, 0.4009042447074329, 0.4009042447074329, 0.4009042447074329, 0.4009042447074329, 0.4009042447074329, 0.40099315466431995, 0.40099315466431995, 0.40099315466431995, 0.40099315466431995, 0.40099315466431995, 0.40099315466431995, 0.40099315466431995, 0.40099315466431995, 0.40099315466431995, 0.40099315466431995, 0.40099315466431995, 0.40099315466431995, 0.40122014889506663, 0.40122014889506663, 0.40122014889506663, 0.40122014889506663, 0.40122014889506663, 0.40122014889506663, 0.40122014889506663, 0.40122014889506663, 0.40122014889506663, 0.40122014889506663, 0.40122014889506663, 0.40122014889506663, 0.40122014889506663, 0.40122014889506663, 0.40122014889506663, 0.40122014889506663, 0.4012

In [135]:
myData = [["orderItemID", "returnShipment"]] + [[str(i+1), str(e[1])] for i,e in enumerate(predictions)]

myFile = open('predict.csv', 'w')
with myFile:
    writer = csv.writer(myFile)
    writer.writerows(myData)
     
print("Writing complete")

Writing complete


In [136]:
# Gauss
clf = GaussianNB()
clf.fit(trainMatrix, labelArray)
predictGauss = clf.predict_proba(testMatrix)

In [137]:
myDataG = [["orderItemID", "returnShipment"]] + [[str(i+1), str(e[1])] for i,e in enumerate(predictGauss)]

myFileG = open('predict.csv', 'w')
with myFileG:
    writer = csv.writer(myFileG)
    writer.writerows(myDataG)
     
print("Writing complete")

Writing complete


In [ ]:
# We want predict rates between 0-0.3 and 0.7-1